# Statistical Modeling

## Last Updated: April 27, 2020

This document was prepared by Leah Ginsberg, a member of the [Ravichandran Research Group](https://www.ravi.caltech.edu/) at [Caltech](http://www.caltech.edu) in collaboration with [Professor Eleftheria Roumeli](https://sites.google.com/uw.edu/roumeli-research-group/) from [University of Washington](https://www.washington.edu/). 

In [1]:
import numpy as np               # general math operations
import pandas as pd              # pandas dataframes
import os                        # for opening .csv files in different folders
import altair as alt             # pretty plotting
import altair_catplot as altcat  # pretty box & whisker plots

import scipy.stats as st           # statistics
import scipy.optimize
import statsmodels.tools.numdiff as smnd
from scipy import signal, fftpack  # filtering
import matplotlib.pyplot as plt    # plotting histogram

import bebi103

import tqdm

import bokeh.plotting
import bokeh.io
bokeh.io.output_notebook()

C:\Users\leahm\anaconda3\lib\site-packages\bebi103\viz.py:37: UserWarning: DataShader import failed with error "No module named 'datashader'".
Features requiring DataShader will not work and you will get exceptions.
  warnings.warn(
C:\Users\leahm\anaconda3\lib\site-packages\bebi103\stan.py:43: UserWarning: Both pystan and cmdstanpy are importable in this environment. As per the cmdstanpy documentation, this is not advised.
  warnings.warn(


Loading BokehJS ...

## Quick Review of Bayesian Modeling

Bayes's theorem tells us the likelihood of certain parameters describing our data, given the data and a prior distribution of the parameters. The equation looks like this:

\begin{align}
&g(\theta | y) = \frac{f(y|\theta) g(\theta)}{f(y)}
\end{align}

where $\theta$ are the parameters that describe our distribution, $y$ is our data, and $g(\theta)$ is our prior distribution, and $f(y)$ is the evidence, which is generally a normalization constant. We will ignore the evidence for our purposes, which will give us something directly proportional to the probability that a selected set of parameters resulted in our data.

First, we have to read in the data that we want to work with and take a look at how it's organized.

In [2]:
df=pd.read_csv('BY2_stiffnesses')
df.head()

,name,max force (uN),location,kinit,deltaF,treatment,details,kret,area,Pos Z zero (um)
0,BY2_in_water/cell114_corr,788.293947,NaN,16.293555,15.371757,water,NaN,310.603424,748.955345,-78.43325
1,BY2_in_water/cell116_corr,796.407908,NaN,1.979630,1.921413,water,NaN,289.081028,1104.217788,-71.63900
2,BY2_in_water/cell117_corr,790.594984,NaN,4.315381,3.939429,water,NaN,416.062647,2959.295861,-61.46350
3,BY2_in_water/cell118_corr,886.403461,NaN,8.235067,7.774974,water,NaN,365.455841,2450.966876,-24.74200
4,BY2_in_water/cell119_corr,897.547332,NaN,14.347290,13.730031,water,NaN,357.142868,3628.969251,-9.12200


In [3]:
df2min = df[df['kinit']>min(df[df['treatment']=='C2']['kinit'])]
df2min[df2min['kinit']==min(df2min[df2min['treatment']=='C2']['kinit'])]

INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


,name,max force (uN),location,kinit,deltaF,treatment,details,kret,area,Pos Z zero (um)
64,BY2_in_C2/cell003_corr,898.611678,cyto,2.188392,2.245624,C2,spreads,305.046353,1164.684169,40.8405


In [5]:
df[df['name']=='BY2_in_C2/cell009_corr']

,name,max force (uN),location,kinit,deltaF,treatment,details,kret,area,Pos Z zero (um)
69,BY2_in_C2/cell009_corr,888.508851,cyto,5.784824,5.886664,C2,spreads,436.637693,2673.367623,37.0845


### Overall Statistics

Let's take a look at some overall statistics to report in the text of the manuscript, starting with the larger groups of all cells in C2-based solutions (GM) and all cells in sorbitol-based solutions (PS).

In [ ]:
ave_GM = np.mean(df[(df['treatment']=='C2')|(df['treatment']=='oryzalin')|(df['treatment']=='LatB')]['kinit'])
std_GM = np.std(df[(df['treatment']=='C2')|(df['treatment']=='oryzalin')|(df['treatment']=='LatB')]['kinit'])
n_GM = len(df[(df['treatment']=='C2')|(df['treatment']=='oryzalin')|(df['treatment']=='LatB')]['kinit'])
stderr_GM = std_GM/np.sqrt(n_GM)
print('The average stiffness in all GM treatments is {:0.2f} +/- {:0.2f} N/m'.format(ave_GM, stderr_GM))

ave_PS = np.mean(df[(df['treatment']=='sorbitol')|(df['treatment']=='oryzalin+sorbitol')|(df['treatment']=='LatB+sorbitol')]['kinit'])
std_PS = np.std(df[(df['treatment']=='sorbitol')|(df['treatment']=='oryzalin+sorbitol')|(df['treatment']=='LatB+sorbitol')]['kinit'])
n_PS = len(df[(df['treatment']=='sorbitol')|(df['treatment']=='oryzalin+sorbitol')|(df['treatment']=='LatB+sorbitol')]['kinit'])
stderr_PS = std_PS/np.sqrt(n_PS)
print('The average stiffness in all PS treatments is {:0.2f} +/- {:0.2f} N/m'.format(ave_PS, stderr_PS))

[KS, pKS] = st.ks_2samp(df[(df['treatment']=='C2')|(df['treatment']=='oryzalin')|(df['treatment']=='LatB')]['kinit'].to_numpy(), 
                           df[(df['treatment']=='sorbitol')|(df['treatment']=='oryzalin+sorbitol')|(df['treatment']=='LatB+sorbitol')]['kinit'].to_numpy())
print('The p-value which separates these two groups, calculated using the nonparametric Kolmogorov-Smirnov test, is {:.2E}'.format(pKS))

[T, pT] = st.ttest_ind(df[(df['treatment']=='C2')|(df['treatment']=='oryzalin')|(df['treatment']=='LatB')]['kinit'].to_numpy(), 
                           df[(df['treatment']=='sorbitol')|(df['treatment']=='oryzalin+sorbitol')|(df['treatment']=='LatB+sorbitol')]['kinit'].to_numpy())
print('The p-value which separates these two groups, calculated using the parametric Student''s t-test, is {:.2E}'.format(pT))

Now let's look at differences within those groups. More specifically, differences between the control cases (pure GM or PS) and solutions with oryzalin to remove MTs or LatB to remove AFs.

In [4]:
ave_PS = np.mean(df[(df['treatment']=='sorbitol')&(df['location']=='cyto')]['kinit'])
std_PS = np.std(df[(df['treatment']=='sorbitol')&(df['location']=='cyto')]['kinit'])
n_PS = len(df[(df['treatment']=='sorbitol')&(df['location']=='cyto')]['kinit'])
stderr_PS = std_PS/np.sqrt(n_PS)
print('The average stiffness in pure PS treatment is {:0.2f} +/- {:0.2f} N/m with n={:0.0f}'.format(ave_PS, stderr_PS, n_PS))

ave_PSMT = np.mean(df[(df['treatment']=='oryzalin+sorbitol')&(df['location']=='cyto')]['kinit'])
std_PSMT = np.std(df[(df['treatment']=='oryzalin+sorbitol')&(df['location']=='cyto')]['kinit'])
n_PSMT = len(df[(df['treatment']=='oryzalin+sorbitol')&(df['location']=='cyto')]['kinit'])
stderr_PSMT = std_PSMT/np.sqrt(n_PSMT)
print('The average stiffness in PS-MT treatment is {:0.2f} +/- {:0.2f} N/m with n={:0.0f}'.format(ave_PSMT, stderr_PSMT, n_PSMT))

ave_PSAF = np.mean(df[(df['treatment']=='LatB+sorbitol')&(df['location']=='cyto')]['kinit'])
std_PSAF = np.std(df[(df['treatment']=='LatB+sorbitol')&(df['location']=='cyto')]['kinit'])
n_PSAF = len(df[(df['treatment']=='LatB+sorbitol')&(df['location']=='cyto')]['kinit'])
stderr_PSAF = std_PSAF/np.sqrt(n_PSAF)
print('The average stiffness in PS-AF treatment is {:0.2f} +/- {:0.2f} N/m with n={:0.0f}'.format(ave_PSAF, stderr_PSAF, n_PSAF))

[KS, pKS] = st.ks_2samp(df[(df['treatment']=='sorbitol')&(df['location']=='cyto')]['kinit'].to_numpy(), 
                           df[(df['treatment']=='oryzalin+sorbitol')&(df['location']=='cyto')]['kinit'].to_numpy())
print('The p-value which separates the PS and PS-MTs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is {:.2E}'.format(pKS))


[KS, pKS] = st.ks_2samp(df[(df['treatment']=='sorbitol')&(df['location']=='cyto')]['kinit'].to_numpy(), 
                           df[(df['treatment']=='LatB+sorbitol')&(df['location']=='cyto')]['kinit'].to_numpy())
print('The p-value which separates the PS and PS-AFs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is {:.2E}'.format(pKS))

[KS, pKS] = st.ks_2samp(df[(df['treatment']=='oryzalin+sorbitol')&(df['location']=='cyto')]['kinit'].to_numpy(), 
                           df[(df['treatment']=='LatB+sorbitol')&(df['location']=='cyto')]['kinit'].to_numpy())
print('The p-value which separates the PS-MTs and PS-AFs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is {:.2E}'.format(pKS))

The average stiffness in pure PS treatment is 2.07 +/- 0.30 N/m with n=28
The average stiffness in PS-MT treatment is 2.28 +/- 0.51 N/m with n=6
The average stiffness in PS-AF treatment is 2.02 +/- 0.24 N/m with n=11
The p-value which separates the PS and PS-MTs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is 4.06E-01
The p-value which separates the PS and PS-AFs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is 3.72E-01
The p-value which separates the PS-MTs and PS-AFs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is 9.99E-01


In [10]:
[T, pT] = st.ttest_ind(df[(df['treatment']=='sorbitol')&(df['location']=='cyto')]['kinit'].to_numpy(), 
                           df[(df['treatment']=='oryzalin+sorbitol')&(df['location']=='cyto')]['kinit'].to_numpy())
print('The p-value which separates the PS and PS-MTs groups, calculated using the parametric Student''s t-test, is {:.2E}'.format(pT))


[T, pT] = st.ttest_ind(df[(df['treatment']=='sorbitol')&(df['location']=='cyto')]['kinit'].to_numpy(), 
                           df[(df['treatment']=='LatB+sorbitol')&(df['location']=='cyto')]['kinit'].to_numpy())
print('The p-value which separates the PS and PS-AFs groups, calculated using the parametric Student''s t-test, is {:.2E}'.format(pT))

[T, pT] = st.ttest_ind(df[(df['treatment']=='oryzalin+sorbitol')&(df['location']=='cyto')]['kinit'].to_numpy(), 
                           df[(df['treatment']=='LatB+sorbitol')&(df['location']=='cyto')]['kinit'].to_numpy())
print('The p-value which separates the PS-MTs and PS-AFs groups, calculated using the parametric Student''s t-test, is {:.2E}'.format(pT))

The p-value which separates the PS and PS-MTs groups, calculated using the parametric Students t-test, is 7.65E-01
The p-value which separates the PS and PS-AFs groups, calculated using the parametric Students t-test, is 9.21E-01
The p-value which separates the PS-MTs and PS-AFs groups, calculated using the parametric Students t-test, is 6.24E-01


In [12]:
ave_GM = np.mean(df[df['treatment']=='C2']['kinit'])
std_GM = np.std(df[df['treatment']=='C2']['kinit'])
n_GM = len(df[df['treatment']=='C2']['kinit'])
stderr_GM = std_GM/np.sqrt(n_GM)
print('The average stiffness in pure GM treatment is {:0.2f} +/- {:0.2f} N/m with n={:0.0f}'.format(ave_GM, stderr_GM, n_GM))

ave_MT = np.mean(df[df['treatment']=='oryzalin']['kinit'])
std_MT = np.std(df[df['treatment']=='oryzalin']['kinit'])
n_MT = len(df[df['treatment']=='oryzalin']['kinit'])
stderr_MT = std_MT/np.sqrt(n_MT)
print('The average stiffness in GM-MT treatment is {:0.2f} +/- {:0.2f} N/m with n={:0.0f}'.format(ave_MT, stderr_MT, n_MT))

ave_AF = np.mean(df[df['treatment']=='LatB']['kinit'])
std_AF = np.std(df[df['treatment']=='LatB']['kinit'])
n_AF = len(df[df['treatment']=='LatB']['kinit'])
stderr_AF = std_AF/np.sqrt(n_AF)
print('The average stiffness in GM-AF treatment is {:0.2f} +/- {:0.2f} N/m with n={:0.0f}'.format(ave_AF, stderr_AF, n_AF))

[KS, pKS] = st.ks_2samp(df[df['treatment']=='C2']['kinit'].to_numpy(), 
                           df[df['treatment']=='oryzalin']['kinit'].to_numpy())
print('The p-value which separates the GM and GM-MTs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is {:.2E}'.format(pKS))

[KS, pKS] = st.ks_2samp(df[df['treatment']=='C2']['kinit'].to_numpy(), 
                           df[df['treatment']=='LatB']['kinit'].to_numpy())
print('The p-value which separates the GM and GM-AFs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is {:.2E}'.format(pKS))
[KS, pKS] = st.ks_2samp(df[df['treatment']=='oryzalin']['kinit'].to_numpy(), 
                           df[df['treatment']=='LatB']['kinit'].to_numpy())
print('The p-value which separates the GM-MTs and GM-AFs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is {:.2E}'.format(pKS))

The average stiffness in pure GM treatment is 8.31 +/- 1.23 N/m with n=19
The average stiffness in GM-MT treatment is 11.98 +/- 1.94 N/m with n=7
The average stiffness in GM-AF treatment is 8.36 +/- 1.37 N/m with n=15
The p-value which separates the GM and GM-MTs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is 1.84E-01
The p-value which separates the GM and GM-AFs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is 1.00E+00
The p-value which separates the GM-MTs and GM-AFs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is 2.28E-01


In [15]:
[T, pT] = st.ttest_ind(df[df['treatment']=='C2']['kinit'].to_numpy(), 
                           df[df['treatment']=='oryzalin']['kinit'].to_numpy())
print('The p-value which separates the GM and GM-MTs groups, calculated using the parametric Student''s t-test, is {:.2E}'.format(pT))

[T, pT] = st.ttest_ind(df[df['treatment']=='C2']['kinit'].to_numpy(), 
                           df[df['treatment']=='LatB']['kinit'].to_numpy())
print('The p-value which separates the GM and GM-AFs groups, calculated using the parametric Student''s t-test, is {:.2E}'.format(pT))
[T, pT] = st.ttest_ind(df[df['treatment']=='oryzalin']['kinit'].to_numpy(), 
                           df[df['treatment']=='LatB']['kinit'].to_numpy())
print('The p-value which separates the GM-MTs and GM-AFs groups, calculated using the parametric Student''s t-test, is {:.2E}'.format(pKS))

The p-value which separates the GM and GM-MTs groups, calculated using the parametric Students t-test, is 1.46E-01
The p-value which separates the GM and GM-AFs groups, calculated using the parametric Students t-test, is 9.80E-01
The p-value which separates the GM-MTs and GM-AFs groups, calculated using the parametric Students t-test, is 2.28E-01


Those numbers give nice summaries of the data, but I want to see all data points on one plot together. To do that, I'll use empirical cumulative distribution functions (ECDFs), which plots each data point (stiffness) against the % of data that is at or below that value.

In [6]:
ECDFs = altcat.catplot(data=df[df['treatment']=='C2'],
               height=350,
               width=450, 
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('red')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[df['treatment']=='LatB'],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('orange')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[df['treatment']=='oryzalin'],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('yellow')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[(df['treatment']=='sorbitol')&(df['location']=='cyto')],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('green')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[(df['treatment']=='LatB+sorbitol')&(df['location']=='cyto')],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('blue')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[(df['treatment']=='oryzalin+sorbitol')&(df['location']=='cyto')],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('purple')},
               transform='ecdf')

ECDFs.configure_text(fontWeight='bold', fontSize=20
).configure_legend(labelFontSize=20, titleFontSize=15
).configure_axis(labelFontSize=20, titleFontSize=20
).configure_mark(color='black'
).configure_point(size=100, strokeWidth=1)

INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


alt.LayerChart(...)

From this plot, we can see the clear separation between treatments in PS-based solutions (in blue, green, and purple), and treatments in GM-based solutions (in red, orange and yellow), as reflected in the p-value we calculated above. There isn't much distinction within those groups, which was also reflected in the previously calculated p-values.

OOPS: In the original submission, I included data from over CW in plasmolyzed cells. We want to remove that data to compare evenly across experiments that include both the response of the CW and the cytoplasm.

In [7]:
df = df[(df['treatment']=='C2')|(df['treatment']=='oryzalin')|(df['treatment']=='LatB')|
        ((df['treatment']=='sorbitol')&(df['location']=='cyto'))|
        ((df['treatment']=='LatB+sorbitol')&(df['location']=='cyto'))|
        ((df['treatment']=='oryzalin+sorbitol')&(df['location']=='cyto'))]

### Box+jitter plots for stiffness

In [8]:
boxjitter = altcat.catplot(df,
               height=350,
               width=450, 
               mark=dict(opacity=1.0, type='point'),
               box_mark=dict(strokeWidth=5, opacity=0.75, color='#abb0ac'),
               whisker_mark=dict(strokeWidth=2, opacity=0.75, color='#bababa'),
               encoding=dict(y=alt.Y('kinit:Q', scale=alt.Scale(domain=(0,22))),
                             x=alt.X('treatment:N'),
#                              # For black&white version
#                              fill=alt.Color('solution:N', scale=alt.Scale(domain=['T87', 'sorbitol', 'water'], range=['#808080', '#000000', '#FFFFFF'])),
#                              # For colored version
                             fill=alt.Color('treatment:N', scale=alt.Scale(domain=['C2', 'LatB', 'LatB+sorbitol','oryzalin','oryzalin+sorbitol','sorbitol'], range=['red', 'orange', 'blue','yellow','purple','green']))),
               transform='jitterbox'
)
boxjitter.configure_text(fontWeight='bold', fontSize=20
).configure_legend(labelFontSize=20, titleFontSize=15
).configure_axis(labelFontSize=20, titleFontSize=20
).configure_mark(color='black'
).configure_point(size=100, strokeWidth=1)

alt.LayerChart(...)

In [9]:
boxjitter = altcat.catplot(df[(df['treatment']=='C2')|(df['treatment']=='LatB')|(df['treatment']=='oryzalin')],
               height=175,
               width=450, 
               mark=dict(opacity=1.0, type='point'),
               box_mark=dict(strokeWidth=5, opacity=0.75, color='#abb0ac'),
               whisker_mark=dict(strokeWidth=2, opacity=0.75, color='#bababa'),
               encoding=dict(x=alt.X('kinit:Q', scale=alt.Scale(domain=(0,22))),
                             y=alt.Y('treatment:N'),
#                              # For black&white version
#                              fill=alt.Color('solution:N', scale=alt.Scale(domain=['T87', 'sorbitol', 'water'], range=['#808080', '#000000', '#FFFFFF'])),
#                              # For colored version
                             fill=alt.Color('treatment:N', scale=alt.Scale(domain=['C2', 'LatB', 'LatB+sorbitol','oryzalin','oryzalin+sorbitol','sorbitol'], range=['red', 'orange', 'blue','yellow','purple','green']))),
               transform='jitterbox'
)
boxjitter.configure_text(fontWeight='bold', fontSize=20
).configure_legend(labelFontSize=20, titleFontSize=15
).configure_axis(labelFontSize=20, titleFontSize=20
).configure_mark(color='black'
).configure_point(size=100, strokeWidth=1)

alt.LayerChart(...)

In [10]:
boxjitter = altcat.catplot(df[(df['treatment']=='sorbitol')|(df['treatment']=='LatB+sorbitol')|(df['treatment']=='oryzalin+sorbitol')],
               height=175,
               width=450, 
               mark=dict(opacity=1.0, type='point'),
               box_mark=dict(strokeWidth=5, opacity=0.75, color='#abb0ac'),
               whisker_mark=dict(strokeWidth=2, opacity=0.75, color='#bababa'),
               encoding=dict(x=alt.X('kinit:Q', scale=alt.Scale(domain=(0,22))),
                             y=alt.Y('treatment:N'),
#                              # For black&white version
#                              fill=alt.Color('solution:N', scale=alt.Scale(domain=['T87', 'sorbitol', 'water'], range=['#808080', '#000000', '#FFFFFF'])),
#                              # For colored version
                             fill=alt.Color('treatment:N', scale=alt.Scale(domain=['C2', 'LatB', 'LatB+sorbitol','oryzalin','oryzalin+sorbitol','sorbitol'], range=['red', 'orange', 'blue','yellow','purple','green']))),
               transform='jitterbox'
)
boxjitter.configure_text(fontWeight='bold', fontSize=20
).configure_legend(labelFontSize=20, titleFontSize=15
).configure_axis(labelFontSize=20, titleFontSize=20
).configure_mark(color='black'
).configure_point(size=100, strokeWidth=1)

alt.LayerChart(...)

### Box + jitter plots for dissipated energy

In [11]:
df = df.dropna()

In [12]:
ECDFs = altcat.catplot(data=df[df['treatment']=='C2'],
               height=350,
               width=450, 
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('area:Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('red')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[df['treatment']=='LatB'],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('area:Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('orange')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[df['treatment']=='oryzalin'],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('area:Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('yellow')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[df['treatment']=='sorbitol'],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('area:Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('green')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[df['treatment']=='LatB+sorbitol'],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('area:Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('blue')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[df['treatment']=='oryzalin+sorbitol'],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('area:Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('purple')},
               transform='ecdf')

ECDFs.configure_text(fontWeight='bold', fontSize=20
).configure_legend(labelFontSize=20, titleFontSize=15
).configure_axis(labelFontSize=20, titleFontSize=20
).configure_mark(color='black'
).configure_point(size=100, strokeWidth=1)

alt.LayerChart(...)

In [13]:
boxjitter = altcat.catplot(df[(df['treatment']=='C2')|(df['treatment']=='LatB')|(df['treatment']=='oryzalin')],
               height=175,
               width=450, 
               mark=dict(opacity=1.0, type='point'),
               box_mark=dict(strokeWidth=5, opacity=0.75, color='#abb0ac'),
               whisker_mark=dict(strokeWidth=2, opacity=0.75, color='#bababa'),
               encoding=dict(x=alt.X('area:Q', scale=alt.Scale(domain=(0,4000))),
                             y=alt.Y('treatment:N'),
#                              # For black&white version
#                              fill=alt.Color('solution:N', scale=alt.Scale(domain=['T87', 'sorbitol', 'water'], range=['#808080', '#000000', '#FFFFFF'])),
#                              # For colored version
                             fill=alt.Color('treatment:N', scale=alt.Scale(domain=['C2', 'LatB', 'LatB+sorbitol','oryzalin','oryzalin+sorbitol','sorbitol'], range=['red', 'orange', 'blue','yellow','purple','green']))),
               transform='jitterbox'
)
boxjitter.configure_text(fontWeight='bold', fontSize=20
).configure_legend(labelFontSize=20, titleFontSize=15
).configure_axis(labelFontSize=20, titleFontSize=20
).configure_mark(color='black'
).configure_point(size=100, strokeWidth=1)

alt.LayerChart(...)

In [14]:
boxjitter = altcat.catplot(df[(df['treatment']=='sorbitol')|(df['treatment']=='LatB+sorbitol')|(df['treatment']=='oryzalin+sorbitol')],
               height=175,
               width=450, 
               mark=dict(opacity=1.0, type='point'),
               box_mark=dict(strokeWidth=5, opacity=0.75, color='#abb0ac'),
               whisker_mark=dict(strokeWidth=2, opacity=0.75, color='#bababa'),
               encoding=dict(x=alt.X('area:Q', scale=alt.Scale(domain=(0,4000))),
                             y=alt.Y('treatment:N'),
#                              # For black&white version
#                              fill=alt.Color('solution:N', scale=alt.Scale(domain=['T87', 'sorbitol', 'water'], range=['#808080', '#000000', '#FFFFFF'])),
#                              # For colored version
                             fill=alt.Color('treatment:N', scale=alt.Scale(domain=['C2', 'LatB', 'LatB+sorbitol','oryzalin','oryzalin+sorbitol','sorbitol'], range=['red', 'orange', 'blue','yellow','purple','green']))),
               transform='jitterbox'
)
boxjitter.configure_text(fontWeight='bold', fontSize=20
).configure_legend(labelFontSize=20, titleFontSize=15
).configure_axis(labelFontSize=20, titleFontSize=20
).configure_mark(color='black'
).configure_point(size=100, strokeWidth=1)

alt.LayerChart(...)

In [16]:
ave_GM = np.mean(df[(df['treatment']=='C2')|(df['treatment']=='oryzalin')|(df['treatment']=='LatB')]['area'])
std_GM = np.std(df[(df['treatment']=='C2')|(df['treatment']=='oryzalin')|(df['treatment']=='LatB')]['area'])
n_GM = len(df[(df['treatment']=='C2')|(df['treatment']=='oryzalin')|(df['treatment']=='LatB')]['area'])
stderr_GM = std_GM/np.sqrt(n_GM)
print('The average energy dissipated in all GM treatments is {:0.2f} +/- {:0.2f} N/m^2'.format(ave_GM, stderr_GM))

ave_PS = np.mean(df[(df['treatment']=='sorbitol')|(df['treatment']=='oryzalin+sorbitol')|(df['treatment']=='LatB+sorbitol')]['area'])
std_PS = np.std(df[(df['treatment']=='sorbitol')|(df['treatment']=='oryzalin+sorbitol')|(df['treatment']=='LatB+sorbitol')]['area'])
n_PS = len(df[(df['treatment']=='sorbitol')|(df['treatment']=='oryzalin+sorbitol')|(df['treatment']=='LatB+sorbitol')]['area'])
stderr_PS = std_PS/np.sqrt(n_PS)
print('The average energy dissipated in all PS treatments is {:0.2f} +/- {:0.2f} N/m^2'.format(ave_PS, stderr_PS))

[KS, pKS] = st.ks_2samp(df[(df['treatment']=='C2')|(df['treatment']=='oryzalin')|(df['treatment']=='LatB')]['area'].to_numpy(), 
                           df[(df['treatment']=='sorbitol')|(df['treatment']=='oryzalin+sorbitol')|(df['treatment']=='LatB+sorbitol')]['area'].to_numpy())
print('The p-value which separates these two groups, calculated using the nonparametric Kolmogorov-Smirnov test, is {:.2E}'.format(pKS))

The average energy dissipated in all GM treatments is 1844.67 +/- 119.00 N/m^2
The average energy dissipated in all PS treatments is 736.18 +/- 177.35 N/m^2
The p-value which separates these two groups, calculated using the nonparametric Kolmogorov-Smirnov test, is 1.46E-07


In [17]:
ave_PS = np.mean(df[(df['treatment']=='sorbitol')&(df['location']=='cyto')]['area'])
std_PS = np.std(df[(df['treatment']=='sorbitol')&(df['location']=='cyto')]['area'])
n_PS = len(df[(df['treatment']=='sorbitol')&(df['location']=='cyto')]['area'])
stderr_PS = std_PS/np.sqrt(n_PS)
print('The average energy dissipated in pure PS treatment is {:0.2f} +/- {:0.2f} N/m^2 with n={:0.0f}'.format(ave_PS, stderr_PS, n_PS))

ave_PSMT = np.mean(df[(df['treatment']=='oryzalin+sorbitol')&(df['location']=='cyto')]['area'])
std_PSMT = np.std(df[(df['treatment']=='oryzalin+sorbitol')&(df['location']=='cyto')]['area'])
n_PSMT = len(df[(df['treatment']=='oryzalin+sorbitol')&(df['location']=='cyto')]['area'])
stderr_PSMT = std_PSMT/np.sqrt(n_PSMT)
print('The average energy dissipated in PS-MT treatment is {:0.2f} +/- {:0.2f} N/m^2 with n={:0.0f}'.format(ave_PSMT, stderr_PSMT, n_PSMT))

ave_PSAF = np.mean(df[(df['treatment']=='LatB+sorbitol')&(df['location']=='cyto')]['area'])
std_PSAF = np.std(df[(df['treatment']=='LatB+sorbitol')&(df['location']=='cyto')]['area'])
n_PSAF = len(df[(df['treatment']=='LatB+sorbitol')&(df['location']=='cyto')]['area'])
stderr_PSAF = std_PSAF/np.sqrt(n_PSAF)
print('The average energy dissipated in PS-AF treatment is {:0.2f} +/- {:0.2f} N/m^2 with n={:0.0f}'.format(ave_PSAF, stderr_PSAF, n_PSAF))

[KS, pKS] = st.ks_2samp(df[(df['treatment']=='sorbitol')&(df['location']=='cyto')]['area'].to_numpy(), 
                           df[(df['treatment']=='oryzalin+sorbitol')&(df['location']=='cyto')]['area'].to_numpy())
print('The p-value which separates the PS and PS-MTs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is {:.2E}'.format(pKS))


[KS, pKS] = st.ks_2samp(df[(df['treatment']=='sorbitol')&(df['location']=='cyto')]['area'].to_numpy(), 
                           df[(df['treatment']=='LatB+sorbitol')&(df['location']=='cyto')]['area'].to_numpy())
print('The p-value which separates the PS and PS-AFs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is {:.2E}'.format(pKS))

[KS, pKS] = st.ks_2samp(df[(df['treatment']=='oryzalin+sorbitol')&(df['location']=='cyto')]['area'].to_numpy(), 
                           df[(df['treatment']=='LatB+sorbitol')&(df['location']=='cyto')]['area'].to_numpy())
print('The p-value which separates the PS-MTs and PS-AFs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is {:.2E}'.format(pKS))

The average energy dissipated in pure PS treatment is 1236.96 +/- 437.70 N/m^2 with n=7
The average energy dissipated in PS-MT treatment is 523.07 +/- 142.00 N/m^2 with n=6
The average energy dissipated in PS-AF treatment is 457.83 +/- 131.65 N/m^2 with n=8
The p-value which separates the PS and PS-MTs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is 9.09E-02
The p-value which separates the PS and PS-AFs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is 2.42E-02
The p-value which separates the PS-MTs and PS-AFs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is 6.37E-01


In [18]:
ave_GM = np.mean(df[df['treatment']=='C2']['area'])
std_GM = np.std(df[df['treatment']=='C2']['area'])
n_GM = len(df[df['treatment']=='C2']['area'])
stderr_GM = std_GM/np.sqrt(n_GM)
print('The average dissipated energy in pure GM treatment is {:0.2f} +/- {:0.2f} N/m^2 with n={:0.0f}'.format(ave_GM, stderr_GM, n_GM))

ave_MT = np.mean(df[df['treatment']=='oryzalin']['area'])
std_MT = np.std(df[df['treatment']=='oryzalin']['area'])
n_MT = len(df[df['treatment']=='oryzalin']['area'])
stderr_MT = std_MT/np.sqrt(n_MT)
print('The average dissipated energy in GM-MT treatment is {:0.2f} +/- {:0.2f} N/m^2 with n={:0.0f}'.format(ave_MT, stderr_MT, n_MT))

ave_AF = np.mean(df[df['treatment']=='LatB']['area'])
std_AF = np.std(df[df['treatment']=='LatB']['area'])
n_AF = len(df[df['treatment']=='LatB']['area'])
stderr_AF = std_AF/np.sqrt(n_AF)
print('The average dissipated energy in GM-AF treatment is {:0.2f} +/- {:0.2f} N/m^2 with n={:0.0f}'.format(ave_AF, stderr_AF, n_AF))

[KS, pKS] = st.ks_2samp(df[df['treatment']=='C2']['area'].to_numpy(), 
                           df[df['treatment']=='oryzalin']['area'].to_numpy())
print('The p-value which separates the GM and GM-MTs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is {:.2E}'.format(pKS))

[KS, pKS] = st.ks_2samp(df[df['treatment']=='C2']['area'].to_numpy(), 
                           df[df['treatment']=='LatB']['area'].to_numpy())
print('The p-value which separates the GM and GM-AFs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is {:.2E}'.format(pKS))
[KS, pKS] = st.ks_2samp(df[df['treatment']=='oryzalin']['area'].to_numpy(), 
                           df[df['treatment']=='LatB']['area'].to_numpy())
print('The p-value which separates the GM-MTs and GM-AFs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is {:.2E}'.format(pKS))

The average dissipated energy in pure GM treatment is 1800.09 +/- 265.24 N/m^2 with n=12
The average dissipated energy in GM-MT treatment is 1799.07 +/- 108.18 N/m^2 with n=7
The average dissipated energy in GM-AF treatment is 1939.57 +/- 58.69 N/m^2 with n=9
The p-value which separates the GM and GM-MTs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is 2.41E-02
The p-value which separates the GM and GM-AFs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is 1.20E-02
The p-value which separates the GM-MTs and GM-AFs groups, calculated using the nonparametric Kolmogorov-Smirnov test, is 5.51E-02


# Generative modeling

I'm going to work with one set of data at the time, starting with the C2 treatment.

In [14]:
x = df.loc[df['treatment']=='C2', 'kinit'].values

# How many data points
len(x)

12

To start, I'm going to assume that these data are drawn from a Gaussian distribution with mean  $\mu=10$ N/m and standard deviation $\sigma=5$ N/m. I'll generate a data set using these parameters through numpy.

In [15]:
x_gen = np.random.normal(10, 5, size=len(x))

Now let's plot the two data sets side by side to compare them.

In [16]:
df_C2 = pd.DataFrame({'kinit':np.concatenate([x,x_gen]), 'type':['orig']*len(x) + ['gen']*len(x)})
altcat.catplot(data=df_C2,
               mark='circle',
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'color': alt.Color('type:N', sort=['orig', 'gen'])},
               transform='ecdf')

alt.Chart(...)

The two data sets appear close, but this could be a lucky draw. Let's draw a lot of sample data sets and see how our data compares to all of them.

In [17]:
ecdf_chart = altcat.catplot(data=df_C2[df_C2['type']=='orig'],
               mark='line',
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'color': alt.Color('type:N', scale=alt.Scale(domain=['gen', 'orig'], range=['#808080', 'blue']))},
               transform='ecdf')
 
for _ in range(100):
    x_gen = np.random.normal(10, 5, size=len(x))
    df_gen = pd.DataFrame({'kinit':x_gen, 'type':['gen']*len(x)})
    ecdf_chart += altcat.catplot(data=df_gen,
               mark=dict(type='line', opacity=0.1),
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'color': alt.Color('type:N', scale=alt.Scale(domain=['gen','orig'], range=['#808080','blue']))},
               transform='ecdf')
    
ecdf_chart

alt.LayerChart(...)

Seems unlikely, but not impossible that this distribution describes our data. Let's use the maximum likelihood estimates (which are the mean and standard deviation for a Gaussian distribution) to see if we can make any improvement.

In [18]:
mu = x.mean()
sigma = x.std()

# Make the draws and make the plot
x_gen = np.random.normal(mu, sigma, size=len(x))


df_C2 = pd.DataFrame({'kinit':np.concatenate([x,x_gen]), 'type':['orig']*len(x) + ['gen']*len(x)})
ecdf_chart = altcat.catplot(data=df_C2[df_C2['type']=='orig'],
               mark='line',
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'color': alt.Color('type:N', scale=alt.Scale(domain=['gen', 'orig'], range=['#808080', 'blue']))},
               transform='ecdf')
 
for _ in range(100):
    x_gen = np.random.normal(mu, sigma, size=len(x))
    df_gen = pd.DataFrame({'kinit':x_gen, 'type':['gen']*len(x)})
    ecdf_chart += altcat.catplot(data=df_gen,
               mark=dict(type='line', opacity=0.1),
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'color': alt.Color('type:N', scale=alt.Scale(domain=['gen','orig'], range=['#808080','blue']))},
               transform='ecdf')
    
ecdf_chart

alt.LayerChart(...)

Still, seems unlikely that this distribution describes our data. Perhaps the spring model will help in finding a more likely generative model. Let's do this same operation for the oryzalin and LatB treatments now.

In [19]:
x = df.loc[df['treatment']=='oryzalin', 'kinit'].values

mu = x.mean()
sigma = x.std()

# Make the draws and make the plot
x_gen = np.random.normal(mu, sigma, size=len(x))


df_ory = pd.DataFrame({'kinit':np.concatenate([x,x_gen]), 'type':['orig']*len(x) + ['gen']*len(x)})
ecdf_chart = altcat.catplot(data=df_ory[df_ory['type']=='orig'],
               mark='line',
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'color': alt.Color('type:N', scale=alt.Scale(domain=['gen', 'orig'], range=['#808080', 'blue']))},
               transform='ecdf')
 
for _ in range(100):
    x_gen = np.random.normal(mu, sigma, size=len(x))
    df_gen = pd.DataFrame({'kinit':x_gen, 'type':['gen']*len(x)})
    ecdf_chart += altcat.catplot(data=df_gen,
               mark=dict(type='line', opacity=0.1),
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'color': alt.Color('type:N', scale=alt.Scale(domain=['gen','orig'], range=['#808080','blue']))},
               transform='ecdf')
    
ecdf_chart

alt.LayerChart(...)

In [20]:
x = df.loc[df['treatment']=='LatB', 'kinit'].values

mu = x.mean()
sigma = x.std()

# Make the draws and make the plot
x_gen = np.random.normal(mu, sigma, size=len(x))


df_latb = pd.DataFrame({'kinit':np.concatenate([x,x_gen]), 'type':['orig']*len(x) + ['gen']*len(x)})
ecdf_chart = altcat.catplot(data=df_latb[df_latb['type']=='orig'],
               mark='line',
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'color': alt.Color('type:N', scale=alt.Scale(domain=['gen', 'orig'], range=['#808080', 'blue']))},
               transform='ecdf')
 
for _ in range(100):
    x_gen = np.random.normal(mu, sigma, size=len(x))
    df_gen = pd.DataFrame({'kinit':x_gen, 'type':['gen']*len(x)})
    ecdf_chart += altcat.catplot(data=df_gen,
               mark=dict(type='line', opacity=0.1),
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'color': alt.Color('type:N', scale=alt.Scale(domain=['gen','orig'], range=['#808080','blue']))},
               transform='ecdf')
    
ecdf_chart

alt.LayerChart(...)

In [21]:
x = df.loc[df['treatment']=='water', 'kinit'].values

mu = x.mean()
sigma = x.std()

# Make the draws and make the plot
x_gen = np.random.normal(mu, sigma, size=len(x))


df_water = pd.DataFrame({'kinit':np.concatenate([x,x_gen]), 'type':['orig']*len(x) + ['gen']*len(x)})
ecdf_chart = altcat.catplot(data=df_water[df_water['type']=='orig'],
               mark='line',
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'color': alt.Color('type:N', scale=alt.Scale(domain=['gen', 'orig'], range=['#808080', 'blue']))},
               transform='ecdf')
 
for _ in range(100):
    x_gen = np.random.normal(mu, sigma, size=len(x))
    df_gen = pd.DataFrame({'kinit':x_gen, 'type':['gen']*len(x)})
    ecdf_chart += altcat.catplot(data=df_gen,
               mark=dict(type='line', opacity=0.1),
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'color': alt.Color('type:N', scale=alt.Scale(domain=['gen','orig'], range=['#808080','blue']))},
               transform='ecdf')
    
ecdf_chart

<ipython-input-21-5a4059675269>:3: RuntimeWarning: Mean of empty slice.
  mu = x.mean()
C:\Users\leahm\anaconda3\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\leahm\anaconda3\lib\site-packages\numpy\core\_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\leahm\anaconda3\lib\site-packages\numpy\core\_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
C:\Users\leahm\anaconda3\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: No objects to concatenate

In [ ]:
x = df.loc[df['treatment']=='sorbitol', 'kinit'].values

mu = x.mean()
sigma = x.std()

# Make the draws and make the plot
x_gen = np.random.normal(mu, sigma, size=len(x))


df_sorb = pd.DataFrame({'kinit':np.concatenate([x,x_gen]), 'type':['orig']*len(x) + ['gen']*len(x)})
ecdf_chart = altcat.catplot(data=df_sorb[df_sorb['type']=='orig'],
               mark='line',
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'color': alt.Color('type:N', scale=alt.Scale(domain=['gen', 'orig'], range=['#808080', 'blue']))},
               transform='ecdf')
 
for _ in range(100):
    x_gen = np.random.normal(mu, sigma, size=len(x))
    df_gen = pd.DataFrame({'kinit':x_gen, 'type':['gen']*len(x)})
    ecdf_chart += altcat.catplot(data=df_gen,
               mark=dict(type='line', opacity=0.1),
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'color': alt.Color('type:N', scale=alt.Scale(domain=['gen','orig'], range=['#808080','blue']))},
               transform='ecdf')
    
ecdf_chart

In [ ]:
x = df.loc[df['treatment']=='oryzalin+sorbitol', 'kinit'].values

mu = x.mean()
sigma = x.std()

# Make the draws and make the plot
x_gen = np.random.normal(mu, sigma, size=len(x))


df_orySorb = pd.DataFrame({'kinit':np.concatenate([x,x_gen]), 'type':['orig']*len(x) + ['gen']*len(x)})
ecdf_chart = altcat.catplot(data=df_orySorb[df_orySorb['type']=='orig'],
               mark='line',
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'color': alt.Color('type:N', scale=alt.Scale(domain=['gen', 'orig'], range=['#808080', 'blue']))},
               transform='ecdf')
 
for _ in range(100):
    x_gen = np.random.normal(mu, sigma, size=len(x))
    df_gen = pd.DataFrame({'kinit':x_gen, 'type':['gen']*len(x)})
    ecdf_chart += altcat.catplot(data=df_gen,
               mark=dict(type='line', opacity=0.1),
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'color': alt.Color('type:N', scale=alt.Scale(domain=['gen','orig'], range=['#808080','blue']))},
               transform='ecdf')
    
ecdf_chart

In [ ]:
x = df.loc[df['treatment']=='LatB+sorbitol', 'kinit'].values

mu = x.mean()
sigma = x.std()

# Make the draws and make the plot
x_gen = np.random.normal(mu, sigma, size=len(x))


df_latbSorb = pd.DataFrame({'kinit':np.concatenate([x,x_gen]), 'type':['orig']*len(x) + ['gen']*len(x)})
ecdf_chart = altcat.catplot(data=df_latbSorb[df_latbSorb['type']=='orig'],
               mark='line',
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'color': alt.Color('type:N', scale=alt.Scale(domain=['gen', 'orig'], range=['#808080', 'blue']))},
               transform='ecdf')
 
for _ in range(100):
    x_gen = np.random.normal(mu, sigma, size=len(x))
    df_gen = pd.DataFrame({'kinit':x_gen, 'type':['gen']*len(x)})
    ecdf_chart += altcat.catplot(data=df_gen,
               mark=dict(type='line', opacity=0.1),
               encoding={'x': alt.X('kinit:Q', scale=alt.Scale(zero=False)),
                         'color': alt.Color('type:N', scale=alt.Scale(domain=['gen','orig'], range=['#808080','blue']))},
               transform='ecdf')
    
ecdf_chart

The oryzalin and LatB treatments seem much more likely to be described by a Gaussian distribution. Maybe the C2 treatment really is a unlikely draw from a Gaussian?

## Creating a Generative Model

Again, I'm going to start by considering only the pure C2 case. I'm going to assume that each measured stiffness is sampled independently and that the variability in each measurement is Gaussian. The first attempt at a generative model for these measurements is:

\begin{align}
&\phi \sim \text{LogNorm}(\ln 10, 0.75),\\[1em]
&\sigma \sim \text{HalfNorm}(0, 10),\\[1em]
&l_i \sim \text{Norm}(\phi, \sigma) \;\forall i.
\end{align}

where I've used the LogNorm and HalfNorm distributions for $\phi$ and $\sigma$ to avoid sampling negative numbers, which would be an unphysical result. Let's take a look at the prior distributions.

In [ ]:
df = df_sorb[df_sorb['type']=='orig'] # considering only C2

In [ ]:
sigma = np.linspace(0, 40, 200)
g = st.halfnorm.pdf(sigma, 0, 10)
df_sigma = pd.DataFrame({'σ': sigma, 'g(σ)': g})
alt.Chart(df_sigma).mark_line().encode(
    x='σ',
    y='g(σ)')

In [ ]:
phi = np.linspace(0, 40, 200)
g = st.lognorm.pdf(phi, 0.75, loc=0, scale=10)

df_phi = pd.DataFrame({'φ': phi, 'g(φ)': g})
alt.Chart(df_phi).mark_line().encode(
    x='φ',
    y='g(φ)')

### Prior predictive checks

Now let's generate samples by drawing from the prior distributions for $\phi$ and $\sigma$ and use those parameter values to generate a data set using the likelihood.

In [ ]:
n_ppc_samples = 1000

# Draw parameters out of the prior
phi = np.random.lognormal(np.log(10), 0.75, size=n_ppc_samples)
sigma = np.abs(np.random.normal(0, 10, size=n_ppc_samples))

# Draw data sets out of the likelihood for each set of prior params
ell = np.array([np.random.normal(ph, sig, size=len(df)) 
                        for ph, sig in zip(phi, sigma)])

df_pred0 = pd.DataFrame({'l': ell[0]})

In [ ]:
ecdf_chart = altcat.catplot(df_pred0,
               mark=dict(type='line', opacity=0.2),
               encoding={'x': alt.X('l:Q', scale=alt.Scale(zero=False))},
               transform='ecdf')
for ell_vals in ell[9::10]:
    df_pred = pd.DataFrame({'l': ell_vals})
    ecdf_chart+=altcat.catplot(df_pred,
               mark=dict(type='line', opacity=0.2),
               encoding={'x': alt.X('l:Q', scale=alt.Scale(zero=False))},
               transform='ecdf')
                         
ecdf_chart

hmmm... still looks like we have a lot of negative stiffness values. Let's calculate what percentage of negative stiffnesses we have from all of the generated data.

In [ ]:
(ell < 0).sum() / (len(df) * n_ppc_samples)

More than 10% of the generated data points are unphysical, so let's try a different model.

### Prior predictive checks, try 2

The standard deviation of stiffness may scale with the stiffness itself. We can express this as

\begin{align}
&\sigma_0 \sim \text{Gamma}(2, 10), \\[1em]
&\sigma = \sigma_0\, \phi.
\end{align}

Then, the prior would look like

In [ ]:
sigma_0 = np.linspace(0, 1, 200)
g = st.gamma.pdf(sigma_0, 2, loc=0, scale=0.1)
df_sigma = pd.DataFrame({'σ': sigma_0, 'g(σ)': g})
alt.Chart(df_sigma).mark_line().encode(
    x='σ',
    y='g(σ)')

And our new complete generative model is

\begin{align}
&\phi \sim \text{LogNorm}(\ln 10, 0.75),\\[1em]
&\sigma_0 \sim \text{Gamma}(2, 10), \\[1em]
&\sigma = \sigma_0\, \phi, \\[1em]
&k_i \sim \text{Norm}(\phi, \sigma) \;\forall i.
\end{align}


Now we can draw parameters from the priors and draw data sets out of the likelihood for each set of prior parameters.

In [ ]:
# Draw parameters out of the prior
phi = np.random.lognormal(np.log(10), 0.75, size=n_ppc_samples)
sigma_0 = np.random.gamma(2, 1/10, size=n_ppc_samples)
sigma = sigma_0 * phi

# Draw data sets out of the likelihood for each set of prior params
ell = np.array([np.random.normal(ph, sig, size=len(df)) 
                        for ph, sig in zip(phi, sigma)])

df_pred0 = pd.DataFrame({'l': ell[0]})

ecdf_chart = altcat.catplot(df_pred0,
               mark=dict(type='line', opacity=0.2),
               encoding={'x': alt.X('l:Q', scale=alt.Scale(zero=False))},
               transform='ecdf')
for ell_vals in ell[9::10]:
    df_pred = pd.DataFrame({'l': ell_vals})
    ecdf_chart+=altcat.catplot(df_pred,
               mark=dict(type='line', opacity=0.2),
               encoding={'x': alt.X('l:Q', scale=alt.Scale(zero=False))},
               transform='ecdf')
                         
ecdf_chart

Yay! Way fewer negative values... let's check again what percentage are negative.

In [ ]:
(ell < 0).sum() / (len(df) * n_ppc_samples)

Less than 1% of the generated data are unphysical, which I can live with.

### Plotting the Posterior

In [ ]:
def log_post_indep_size(params, ell):
    """Log posterior for independent size spindle model"""
    # Make sure parameters are physical
    if (params < 0).any():
        return -np.inf
    
    # Unpack parameters
    phi, sigma_0 = params
    
    # Prior on phi
    log_prior = st.lognorm.logpdf(phi, 0.75, loc=0, scale=20)

    # Prior on sigma_0
    log_prior += st.gamma.logpdf(sigma_0, 2, loc=0, scale=1/10)
        
    # Likelihood
    log_like = np.sum(st.norm.logpdf(ell, phi, sigma_0*phi))
    
    return log_prior + log_like

In [ ]:
# Set up plotting range
phi = np.linspace(7.5, 11.5, 200)
sigma0 = np.linspace(0.3, 1.0, 200)
PHI, SIGMA0 = np.meshgrid(phi, sigma0)

In [ ]:
# Slice out spindle lengths as numpy array for speed
ell = df['kinit'].values

# Compute log posterior
LOG_POST = np.empty_like(PHI)
for i in tqdm.tqdm(range(PHI.shape[0])):
    for j in range(PHI.shape[1]):
        LOG_POST[i, j] = log_post_indep_size(
                            np.array([PHI[i, j], SIGMA0[i,j]]), ell)
        
# Exponentiate. Ignore normalization, so set max LOG_POST to zero
POST = np.exp(LOG_POST - LOG_POST.max())

In [ ]:
# Make plot
p = bebi103.viz.contour(PHI,
                        SIGMA0,
                        POST,
                        x_axis_label='φ (N/m)',
                        y_axis_label='σ₀',
                        title='Posterior for independent size model',
                        overlaid=True)
bokeh.io.show(p)

## Estimation of MAP Parameters

In [ ]:
def neg_log_post_indep_size(params, ell):
    return -log_post_indep_size(params, ell)

In [ ]:
params_0 = np.array([8, 0.55]) # initial guess

In [ ]:
# Extra arguments as a tuple (note the trailing comma; must be tuple)
args = (df['kinit'].values,)

# Compute the MAP
res = scipy.optimize.minimize(neg_log_post_indep_size, 
                              params_0,
                              args=args,
                              method='powell')

# Extract optimal parameters
popt = res.x

# For convenience...
phi_MAP, sigma_0_MAP = popt

# Print results
print("""
Most probable parameters
------------------------
φ = {0:.2f} N/m
σ₀ = {1:.3f}
""".format(phi_MAP, sigma_0_MAP))

## Gaussian approximation of the posterior

In [ ]:
hes = smnd.approx_hess(popt, log_post_indep_size, args=args)

In [ ]:
# Compute the covariance matrix
cov = -np.linalg.inv(hes)

# Look at it
cov

The diagonal terms give the approximate variance in the regression parameters. The off-diagonal terms give the covariance, which describes how the parameters relate to each other. Nonzero convariance indicates that they are not completely independent.

In [ ]:
# Report results
print("""
Results for C2 treatment overall spring model (≈ 68% of total probability)
--------------------------------------------------------------------------
φ = {0:.2f} ± {1:.2f} N/m
σ₀ = {2:.3f} ± {3:.3f}
""".format(phi_MAP, np.sqrt(cov[0,0]), sigma_0_MAP, np.sqrt(cov[1,1])))

In [ ]:
# Multivariate Gaussian
post_gauss = np.empty_like(PHI)
for i in tqdm.tqdm(range(PHI.shape[0])):
    for j in range(PHI.shape[1]):
        post_gauss[i, j] = st.multivariate_normal.pdf(
                            (PHI[i, j], SIGMA0[i, j]), popt, cov)
        
# Make contours
p = bebi103.viz.contour(PHI,
                        SIGMA0,
                        POST,
                        x_axis_label='γ',
                        y_axis_label='φ (µm)')
p = bebi103.viz.contour(PHI,
                        SIGMA0,
                        post_gauss,
                        line_color='blue',
                        p=p)
bokeh.io.show(p)

## Hypothesis Testing

To test for differences in the stiffness of cells under various conditions, I will use 3 different tests; one-way ANOVA, Student's t-test, and the two-sample KS test.

**ANOVA** stands for "Analysis of Variance" and is an omnibus test, meaning it tests for a difference overall between all groups. The one-way ANOVA, also referred to as one factor ANOVA, is a parametric test used to test for a statistically significant difference of an outcome between 3 or more groups. Since it is an omnibus test, it tests for a difference overall, i.e. at least one of the groups is statistically significantly different than the others. However, if the ANOVA is significant one cannot tell which group is different. In order to tell which group is different, one has to conduct planned or post-hoc comparisons. As with all parametric tests, there are certain conditions that need to be met in order for the test results to be considered reliable.

1. The samples are independent.
2. Each sample is from a normally distributed population.
3. The population standard deviations of the groups are all equal. This property is known as *homoscedasticity*.

**Student's t-test** is another parametric test which is commonly used to test the simple hypothesis (that the means of two populations are the same). The two-sided t-test is a parametric used to test for a statistically significant difference between the means of two groups. In order to use Student's t-test, the sample mean of each group must be normally distributed (or equivalently, the central limit theorem is applicable to each sampling group, but the t-test does not require large N like the central limit theorem). It is most common to use the t-test if the data are modeled as a sample from the normal (Gaussian) distribution, and the sample size is small.

The **two-sample Kolmogorov-Smirnov test** is a nonparametric test of the equality of two empirical cumulative distribution functions. The two-sample K–S test is one of the most useful and general nonparametric methods for comparing two samples, as it is sensitive to differences in both location and shape of the empirical cumulative distribution functions of the two samples.

In all data sets, we want to know if the true mean of the distribution we are drawing from (the true initial stiffness and true dissipated energy with experimental noise) are the same. We can test the hypothesis that these two means are the same, and then use p-values to determine how much evidence we have against this hypothesis. Specifically, we want to test

$H_0: \Delta \mu = 0$ vs. $H_1: \Delta \mu \neq 0$

where $\Delta \mu = \mu_1 - \mu_2$. Let $X_1, ..., X_n$ and $Y_1, ..., Y_m$ be two independent data sets. The plug-in estimates of the means are: $\hat{\mu}_1 = \bar{X}_n$ and $\hat{\mu}_2 = \bar{Y}_m$. A nonparametric estimate of $\Delta \mu$ is thus

$\hat{\Delta \mu} = \bar{X}_n - \bar{Y}_m$

The standard error of $\hat{\Delta \mu}$ is

$\text{se}^2 = \text{se}^2[\bar{X}_n] + \text{se}^2[\bar{Y}_m] = \frac{\sigma_1^2}{n} + \frac{\sigma_2^2}{m}$

where $\sigma_1^2$ and $\sigma_2^2$ are the true variances. They can be estimated by

$\hat{se}^2 = \frac{s_1^2}{n} + \frac{s_2^2}{m}$

where $s_1^2$ and $s_2^2$ are the sample variances. 

In [ ]:
# overall statistics
rp.summary_cont(df['kinit'])

In [ ]:
# df.loc[df['treatment']=='sorbitol',:]=df[(df['treatment']=='sorbitol')&(df['location']=='cyto')]
# df.loc[df['treatment']=='LatB+sorbitol',:]=df[(df['treatment']=='LatB+sorbitol')&(df['location']=='cyto')]

In [ ]:
# group statistics
rp.summary_cont(df[df['location']=='cyto']['area'].groupby(df['treatment']))

In [ ]:
# p-values from KS test
[KS, pKS] = stats.ks_2samp(df[(df['max force (uN)']>700)&(df['location']=='cyto')&(df['treatment']=='sorbitol')]['area'].to_numpy(), 
                df[(df['max force (uN)']>700)&(df['location']=='cyto')&(df['treatment']=='LatB+sorbitol')]['area'].to_numpy())
pKS

In [ ]:
# p-values from Student's t-test
[TS, pTS] = stats.ttest_ind(df[(df['max force (uN)']>700)&(df['location']=='cyto')&(df['treatment']=='oryzalin+sorbitol')]['area'].to_numpy(), 
                df[(df['max force (uN)']>700)&(df['location']=='cyto')&(df['treatment']=='LatB+sorbitol')]['area'].to_numpy())
pTS

In [ ]:
# # only take fits with max force over 700uN
# df_kref = df_kref[df_kref['max force (uN)']>700]

# sample means
kinit_bar_water = np.mean(df_kref.loc[df_kref['treatment']=='water','kinit']) # average kinit for water treatment
kinit_bar_C2 = np.mean(df_kref.loc[df_kref['treatment']=='C2','kinit']) # average kinit for C2 treatment
kinit_bar_ory = np.mean(df_kref.loc[df_kref['treatment']=='oryzalin','kinit']) # average kinit for oryzalin treatment
kinit_bar_latb = np.mean(df_kref.loc[df_kref['treatment']=='LatB','kinit']) # average kinit for LatB treatment
kinit_bar_sorb = np.mean(df_kref.loc[df_kref['treatment']=='sorbitol','kinit']) # average kinit for sorbitol treatment
kinit_bar_ory_sorb = np.mean(df_kref.loc[df_kref['treatment']=='oryzalin+sorbitol','kinit']) # average kinit for oryzalin+sorbitol treatment
kinit_bar_latb_sorb = np.mean(df_kref.loc[df_kref['treatment']=='LatB+sorbitol','kinit']) # average kinit for LatB+sorbitol treatment

# a_bar_water = np.mean(df_area.loc[df_area['treatment']=='water','area']) # average area between indenting and retraction curves for water treatment
# a_bar_C2 = np.mean(df_area.loc[df_area['treatment']=='C2','area']) # average area between indenting and retraction curves for C2 treatment
# a_bar_ory = np.mean(df_area.loc[df_area['treatment']=='oryzalin','area']) # average area between indenting and retraction curves for oryzalin treatment
# a_bar_latb = np.mean(df_area.loc[df_area['treatment']=='LatB','area']) # average area between indenting and retraction curves for LatB treatment
# a_bar_sorb = np.mean(df_area.loc[df_area['treatment']=='sorbitol','area']) # average area between indenting and retraction curves for sorbitol treatment
# a_bar_ory_sorb = np.mean(df_area.loc[df_area['treatment']=='oryzalin+sorbitol','area']) # average area between indenting and retraction curves for oryzalin+sorbitol treatment
# a_bar_latb_sorb = np.mean(df_area.loc[df_area['treatment']=='LatB+sorbitol','area']) # average area between indenting and retraction curves for LatB+sorbitol treatment

# maxe_bar_water = np.mean(df_area.loc[df_area['treatment']=='water','max dissipated energy']) # average max dissipated energy for water treatment
# maxe_bar_C2 = np.mean(df_area.loc[df_area['treatment']=='C2','max dissipated energy']) # average max dissipated energy for C2 treatment
# maxe_bar_ory = np.mean(df_area.loc[df_area['treatment']=='oryzalin','max dissipated energy']) # average max dissipated energy for oryzalin treatment
# maxe_bar_latb = np.mean(df_area.loc[df_area['treatment']=='LatB','max dissipated energy']) # average max dissipated energy for LatB treatment
# maxe_bar_sorb = np.mean(df_area.loc[df_area['treatment']=='sorbitol','max dissipated energy']) # average max dissipated energyfor sorbitol treatment
# maxe_bar_ory_sorb = np.mean(df_area.loc[df_area['treatment']=='oryzalin+sorbitol','max dissipated energy']) # average max dissipated energy for oryzalin+sorbitol treatment
# maxe_bar_latb_sorb = np.mean(df_area.loc[df_area['treatment']=='LatB+sorbitol','max dissipated energy']) # average max dissipated energy for LatB+sorbitol treatment

# sample sizes
n_water = len(df_kref.loc[df_kref['treatment']=='water','kinit'])
n_C2 = len(df_kref.loc[df_kref['treatment']=='C2','kinit'])
n_ory = len(df_kref.loc[df_kref['treatment']=='oryzalin','kinit'])
n_latb = len(df_kref.loc[df_kref['treatment']=='LatB','kinit'])
n_sorb = len(df_kref.loc[df_kref['treatment']=='sorbitol','kinit'])
n_ory_sorb = len(df_kref.loc[df_kref['treatment']=='oryzalin+sorbitol','kinit'])
n_latb_sorb = len(df_kref.loc[df_kref['treatment']=='LatB+sorbitol','kinit'])

# sample variances
var_kinit_water = np.var(df_kref.loc[df_kref['treatment']=='water','kinit']) # sample variance of kinit for water treatment
var_kinit_C2 = np.var(df_kref.loc[df_kref['treatment']=='C2','kinit']) # sample variance of kinit for C2 treatment
var_kinit_ory = np.var(df_kref.loc[df_kref['treatment']=='oryzalin','kinit']) # sample variance of kinit for oryzalin treatment
var_kinit_latb = np.var(df_kref.loc[df_kref['treatment']=='LatB','kinit']) # sample variance of kinit for LatB treatment
var_kinit_sorb = np.var(df_kref.loc[df_kref['treatment']=='sorbitol','kinit']) # sample variance of kinit for C2 treatment
var_kinit_ory_sorb = np.var(df_kref.loc[df_kref['treatment']=='oryzalin+sorbitol','kinit']) # sample variance of kinit for oryzalin treatment
var_kinit_latb_sorb = np.var(df_kref.loc[df_kref['treatment']=='LatB+sorbitol','kinit']) # sample variance of kinit for LatB treatment

# var_a_water = np.var(df_area.loc[df_area['treatment']=='water','area']) # sample variance of area for water treatment
# var_a_C2 = np.var(df_area.loc[df_area['treatment']=='C2','area']) # sample variance of area for C2 treatment
# var_a_ory = np.var(df_area.loc[df_area['treatment']=='oryzalin','area']) # sample variance of area for oryzalin treatment
# var_a_latb = np.var(df_area.loc[df_area['treatment']=='LatB','area']) # sample variance of area for LatB treatment
# var_a_sorb = np.var(df_area.loc[df_area['treatment']=='sorbitol','area']) # sample variance of area for sorbitol treatment
# var_a_ory_sorb = np.var(df_area.loc[df_area['treatment']=='oryzalin+sorbitol','area']) # sample variance of area for oryzalin+sorbitol treatment
# var_a_latb_sorb = np.var(df_area.loc[df_area['treatment']=='LatB+sorbitol','area']) # sample variance of area for LatB+sorbitol treatment

# var_maxe_water = np.var(df_area.loc[df_area['treatment']=='water','max dissipated energy']) # sample variance of area for water treatment
# var_maxe_C2 = np.var(df_area.loc[df_area['treatment']=='C2','max dissipated energy']) # sample variance of area for C2 treatment
# var_maxe_ory = np.var(df_area.loc[df_area['treatment']=='oryzalin','max dissipated energy']) # sample variance of area for oryzalin treatment
# var_maxe_latb = np.var(df_area.loc[df_area['treatment']=='LatB','max dissipated energy']) # sample variance of area for LatB treatment
# var_maxe_sorb = np.var(df_area.loc[df_area['treatment']=='sorbitol','max dissipated energy']) # sample variance of area for sorbitol treatment
# var_maxe_ory_sorb = np.var(df_area.loc[df_area['treatment']=='oryzalin+sorbitol','max dissipated energy']) # sample variance of area for oryzalin+sorbitol treatment
# var_maxe_latb_sorb = np.var(df_area.loc[df_area['treatment']=='LatB+sorbitol','max dissipated energy']) # sample variance of area for LatB+sorbitol treatment

In [ ]:
# organize into dataframes
treatments = ['water','C2','oryzalin','LatB','sorbitol','oryzalin+sorbitol','LatB+sorbitol']
kinit_bars = [kinit_bar_water, kinit_bar_C2, kinit_bar_ory, kinit_bar_latb, kinit_bar_sorb, kinit_bar_ory_sorb, kinit_bar_latb_sorb]
ns = [n_water, n_C2, n_ory, n_latb, n_sorb, n_ory_sorb, n_latb_sorb]
kinit_vars = [var_kinit_water, var_kinit_C2, var_kinit_ory, var_kinit_latb, var_kinit_sorb, var_kinit_ory_sorb, var_kinit_latb_sorb]
# a_bars = [a_bar_water, a_bar_C2, a_bar_ory, a_bar_latb, a_bar_sorb, a_bar_ory_sorb, a_bar_latb_sorb]
# a_vars = [var_a_water, var_a_C2, var_a_ory, var_a_latb, var_a_sorb, var_a_ory_sorb, var_a_latb_sorb]
# maxe_bars = [maxe_bar_water, maxe_bar_C2, maxe_bar_ory, maxe_bar_latb, maxe_bar_sorb, maxe_bar_ory_sorb, maxe_bar_latb_sorb]
# maxe_vars = [var_maxe_water, var_maxe_C2, var_maxe_ory, var_maxe_latb, var_maxe_sorb, var_maxe_ory_sorb, var_maxe_latb_sorb]

# df_stats = pd.DataFrame({'treatment': treatments, 'n': ns, 'kinit mean': kinit_bars, 'kinit variance': kinit_vars, 'area mean': a_bars, 'area variance': a_vars, 'max diss energy mean': maxe_bars, 'max diss energy variance': maxe_vars})
df_stats = pd.DataFrame({'treatment': treatments, 'n': ns, 'kinit mean': kinit_bars, 'kinit variance': kinit_vars})
df_stats

### Student's t-test

The t-test is a parametric test which is commonly used if the data is modeled as a sample from the normal distribution and the sample size is small. Both of those conditions are satisfied for our data. (When $n \approx 30$ the t-test is identical to the Wald test, but our $n$ is about 20.)

The size $\alpha$ t-test rejects $H_0$ when

$\left| \frac{\bar{X}_n - \bar{Y}_m}{\sqrt{\frac{s_1^2}{n}+\frac{s_2^2}{m}}}\right| > t_{n-1,1-\frac{\alpha}{2}}$

where $t_{k,\alpha}$ is Student's t-distribution with $n-1$ degrees of freedom.

Below are the p-values comparing the means of every treatment to each other using the initial stiffness calculation.

In [ ]:
# Test statistic
print('                   |        water      |         C2        |      oryzalin     |        LatB       |      sorbitol     | oryzalin+sorbitol |   LatB+sorbitol   ')
print('---------------------------------------------------------------------------------------------------------------------------------------------------------------')

for i in range(len(df_stats)):
    
    # select 1st treatment for comparison
    comp1 = df_stats.iloc[i]
    line = '{:^19}'.format(comp1['treatment'])
    for j in range(len(df_stats)):
        
        # select 2nd treatment for comparison
        comp2 = df_stats.iloc[j]

        # calculate test statistics
        kinit_stat = abs((comp1['kinit mean']-comp2['kinit mean']) / np.sqrt(comp1['kinit variance']/comp1['n'] + comp2['kinit variance']/comp2['n']))
#         area_stat = abs((comp1['area mean']-comp2['area mean']) / np.sqrt(comp1['area variance']/comp1['n'] + comp2['area variance']/comp2['n']))
        # degrees of freedom
        df = (comp1['kinit variance']/comp1['n'] + comp2['kinit variance']/comp2['n'])**2/((comp1['kinit variance']/comp1['n'])**2/(comp1['n']-1)+(comp2['kinit variance']/comp2['n'])**2/(comp2['n']-1))
        # two-sided pvalue=prob(abs(t)>tt)
        line += '|{:19.2e}'.format(stats.t.sf(np.abs(kinit_stat), df)*2)
#         print('The p-value for area comparing ' + comp1['treatment'] + ' and ' + comp2['treatment'] + ' treatments: ' + str(stats.t.pdf(area_stat, comp1['n'])))
    print(line)

Below are the p-values comparing the means of every treatment to each other using the area calculation.

In [ ]:
# Test statistic
print('                   |         C2        |      oryzalin     |        LatB       |      sorbitol     | oryzalin+sorbitol |   LatB+sorbitol   ')
print('-------------------------------------------------------------------------------------------------------------------------------------------')

for i in range(len(df_stats)):
    
    # select 1st treatment for comparison
    comp1 = df_stats.iloc[i]
    line = '{:^19}'.format(comp1['treatment'])
    for j in range(len(df_stats)):
        
        # select 2nd treatment for comparison
        comp2 = df_stats.iloc[j]

        # calculate test statistics
        area_stat = abs((comp1['area mean']-comp2['area mean']) / np.sqrt(comp1['area variance']/comp1['n'] + comp2['area variance']/comp2['n']))
        
        # two-sided pvalue=prob(abs(t)>tt)
        line += '|{:19.2e}'.format(stats.t.sf(np.abs(area_stat), comp1['n']-1)*2)
    print(line)

In [ ]:
# Test statistic
print('                   |         C2        |      oryzalin     |        LatB       |      sorbitol     | oryzalin+sorbitol |   LatB+sorbitol   ')
print('-------------------------------------------------------------------------------------------------------------------------------------------')

for i in range(len(df_stats)):
    
    # select 1st treatment for comparison
    comp1 = df_stats.iloc[i]
    line = '{:^19}'.format(comp1['treatment'])
    for j in range(len(df_stats)):
        
        # select 2nd treatment for comparison
        comp2 = df_stats.iloc[j]

        # calculate test statistics
        maxe_stat = abs((comp1['max diss energy mean']-comp2['max diss energy mean']) / np.sqrt(comp1['max diss energy variance']/comp1['n'] + comp2['max diss energy variance']/comp2['n']))
#         area_stat = abs((comp1['area mean']-comp2['area mean']) / np.sqrt(comp1['area variance']/comp1['n'] + comp2['area variance']/comp2['n']))
        
        # two-sided pvalue=prob(abs(t)>tt)
        line += '|{:19.2e}'.format(stats.t.sf(np.abs(maxe_stat), comp1['n']-1)*2)
#         print('The p-value for area comparing ' + comp1['treatment'] + ' and ' + comp2['treatment'] + ' treatments: ' + str(stats.t.pdf(area_stat, comp1['n'])))
    print(line)

For reference, the following scale of p-values are typically used:
<ul>
  <li>p < 0.01 : very strong evidence against $H_0$</li>
  <li>p $\epsilon$ (0.01,0.05) : strong evidence against $H_0$</li>
  <li>p $\epsilon$ (0.05,0.1) : weak evidence against $H_0$</li>
  <li>p > 0.1 : little or no evidence against $H_0$</li>
</ul>

Let's see what happens if we just compare treatments without sorbitol to those with sorbitol.

### One-Way ANOVA

In [ ]:
df_kref[df_kref['treatment']=='C2']['kinit']

In [ ]:
stats.f_oneway(df_kref[df_kref['treatment']=='C2']['kinit'], df_kref[df_kref['treatment']=='oryzalin']['kinit'], df_kref[df_kref['treatment']=='LatB']['kinit'])

### KS-Test

In [ ]:
# Test statistic
print('                   |         C2        |      oryzalin     |        LatB       |      sorbitol     | oryzalin+sorbitol |   LatB+sorbitol   ')
print('-------------------------------------------------------------------------------------------------------------------------------------------')

for i in range(len(df_stats)):
    
    # select 1st treatment for comparison
    comp1 = df_stats.iloc[i]
    line = '{:^19}'.format(comp1['treatment'])
    for j in range(len(df_stats)):
        
        # select 2nd treatment for comparison
        comp2 = df_stats.iloc[j]

        # calculate test statistics
        [KS, pKS] = stats.ks_2samp(df_kref[df_kref['treatment']==comp1['treatment']]['kinit'].to_numpy(), 
                df_kref[df_kref['treatment']==comp2['treatment']]['kinit'].to_numpy())

        # 1-alpha/2 quantile of normal distribution
        line += '|{:19.2e}'.format(pKS)
#         print('The p-value for area comparing ' + comp1['treatment'] + ' and ' + comp2['treatment'] + ' treatments: ' + str(stats.t.pdf(area_stat, comp1['n'])))
    print(line)

In [ ]:
# Test statistic
print('                   |         C2        |      oryzalin     |        LatB       |      sorbitol     | oryzalin+sorbitol |   LatB+sorbitol   ')
print('-------------------------------------------------------------------------------------------------------------------------------------------')

for i in range(len(df_stats)):
    
    # select 1st treatment for comparison
    comp1 = df_stats.iloc[i]
    line = '{:^19}'.format(comp1['treatment'])
    for j in range(len(df_stats)):
        
        # select 2nd treatment for comparison
        comp2 = df_stats.iloc[j]

        # calculate test statistics
        [KS, pKS] = stats.ks_2samp(df_area[df_area['treatment']==comp1['treatment']]['area'].to_numpy(), 
                df_area[df_area['treatment']==comp2['treatment']]['area'].to_numpy())

        # 1-alpha/2 quantile of normal distribution
        line += '|{:19.2e}'.format(pKS)
#         print('The p-value for area comparing ' + comp1['treatment'] + ' and ' + comp2['treatment'] + ' treatments: ' + str(stats.t.pdf(area_stat, comp1['n'])))
    print(line)